In [ ]:
import shutil
import os
import xml_to_dict as xtd
import random


# Process CVAT labelled data for YOLOv8 training

The provided data by ASPL was labelled through rotated bounding box. However, rotated/oriented bounding box remains to be a frontier of research and there is not a lot of format supporting it.

Thus, this is an attempt to convert from LabelMe format to YOLO OBB/Segmentation format.


In [ ]:
def process_cvat():
    search_dir = os.path.abspath('./datasets/aspl_images_post_labelled\images')

    obj_idx_dict = {
            "Main Pad": 0,
            "LED Pads": 1,
            "Main Chip": 2,
            "Left LED Chip": 3,
            "Right LED Chip": 4,
            "Main PCB": 5
        }

    parser = xtd.XMLtoDict()

    for root, _, files in os.walk(search_dir):

        for file in files:

            if file.endswith(".xml"):

                file_path = os.path.join(root, file)
                file_name = file.split(".")[0]

                with open(file_path, 'r') as f:
                    content = f.read()
                
                xml_dict = parser.parse(content)['annotation']

                img_x = int(xml_dict['imagesize']['ncols'])
                img_y = int(xml_dict['imagesize']['nrows'])

                with open(os.path.join(root, file_name + '.txt'), 'w') as f:

                    for obj in xml_dict['object']:
                        obj_name = obj['name']
                        verts = []

                        for pt in obj['polygon']['pt']:
                            verts.append(f"{float(pt['x']) / img_x:.6f}")
                            verts.append(f"{float(pt['y']) / img_y:.6f}")
                    
                        f.write(f'{str(obj_idx_dict[obj_name])} {" ".join(verts)}\n')
    
process_cvat()


In [ ]:
def split_cvat():
    search_dir = os.path.abspath('./datasets/aspl_images/images/Train')
    val_dir = os.path.abspath('./datasets/aspl_images/images/Val')
    
    if not os.path.exists(val_dir):
        os.makedirs(val_dir)

    for file in os.listdir(search_dir):
        if file.endswith(".txt"):

            file_name = file.split(".")[0]

            if random.random() < 0.2:
                shutil.move(os.path.join(search_dir, file), val_dir)
                shutil.move(os.path.join(search_dir, file_name + '.jpg'), val_dir)

split_cvat()


# Process Rendered data for YOLOv8 training

Rendered data is already compatible with YOLO OBB/Segmentation format.

Just get a list of all files, and copy both the label and the image into train/test folders.


In [ ]:
def rendered_select():
    train_dir = os.path.abspath('./datasets/rendered/train')
    test_dir = os.path.abspath('./datasets/rendered/test')
    backup_dir = os.path.abspath('./datasets/rendered/backup')

    # from backup folder:
    # select 500 good, 360 bad, 240 mirage for train
    # select 120 good, 85 bad, 60 mirage for test
    # move the correspnding txt files too

    file_list = os.listdir(backup_dir)
    random.shuffle(file_list)

    good_count = 0
    bad_count = 0
    mirage_count = 0

    for file in file_list:
        if file.endswith(".png"):
            if "good" in file:
                if good_count < 500:
                    shutil.move(os.path.join(backup_dir, file), train_dir)
                    shutil.move(os.path.join(backup_dir, file.split(".")[0] + ".txt"), train_dir)
                    good_count += 1
                elif good_count < 620:
                    shutil.move(os.path.join(backup_dir, file), test_dir)
                    shutil.move(os.path.join(backup_dir, file.split(".")[0] + ".txt"), test_dir)
                    good_count += 1
            elif "bad" in file:
                if bad_count < 360:
                    shutil.move(os.path.join(backup_dir, file), train_dir)
                    shutil.move(os.path.join(backup_dir, file.split(".")[0] + ".txt"), train_dir)
                    bad_count += 1
                elif bad_count < 445:
                    shutil.move(os.path.join(backup_dir, file), test_dir)
                    shutil.move(os.path.join(backup_dir, file.split(".")[0] + ".txt"), test_dir)
                    bad_count += 1
            elif "mirage" in file:
                if mirage_count < 240:
                    shutil.move(os.path.join(backup_dir, file), train_dir)
                    shutil.move(os.path.join(backup_dir, file.split(".")[0] + ".txt"), train_dir)
                    mirage_count += 1
                elif mirage_count < 300:
                    shutil.move(os.path.join(backup_dir, file), test_dir)
                    shutil.move(os.path.join(backup_dir, file.split(".")[0] + ".txt"), test_dir)
                    mirage_count += 1

rendered_select()


In [ ]:
def process_rendered():
    img_dir = os.path.abspath('./datasets/rendered')
    lbl_dir = os.path.abspath('./datasets/rendered/labels')

    test_dir = os.path.abspath('./datasets/rendered/test')
    train_dir = os.path.abspath('./datasets/rendered/train')

    if not os.path.exists(test_dir):
        os.makedirs(test_dir)

    if not os.path.exists(train_dir):
        os.makedirs(train_dir)

    file_list = [file for file in os.listdir(img_dir) if file.endswith('.png')]
    random.shuffle(file_list)
    n_files = len(file_list)

    for i, item in enumerate(file_list):
        
        item_name = item.split('.')[0]
        img_path = os.path.join(img_dir, item)
        lbl_path = os.path.join(lbl_dir, item_name + '.txt')

        if i < n_files * 0.2:
            shutil.move(img_path, os.path.join(test_dir, item))
            shutil.move(lbl_path, os.path.join(test_dir, item_name + '.txt'))
        else:
            shutil.move(img_path, os.path.join(train_dir, item))
            shutil.move(lbl_path, os.path.join(train_dir, item_name + '.txt'))

        if (i - 1) % 100 == 0 or i == n_files - 1:
            print(f"{i+1} / {n_files} files processed")
    
process_rendered()


In [ ]:
def fix_labels():
    search_dir = os.path.abspath('./datasets/rendered')
    for root, _, files in os.walk(search_dir):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                labels = []

                with open(file_path, 'r') as f:
                    labels = f.read().splitlines()

                new_labels = []
                for label in labels:
                    label = label.split(' ')
                    label = [ float(x) for x in label ]
                    label[0] = int(label[0])

                    ori_label = label.copy()

                    if label[0] == 2:
                        label[2] = 1-ori_label[2]
                        label[4] = 1-ori_label[4]
                        label[6] = 1-ori_label[6]
                        label[8] = 1-ori_label[8]
                    else:
                        label[1] = ori_label[1]
                        label[2] = 1-ori_label[2]
                        label[3] = ori_label[5]
                        label[4] = 1-ori_label[6]
                        label[5] = ori_label[7]
                        label[6] = 1-ori_label[8]
                        label[7] = ori_label[3]
                        label[8] = 1-ori_label[4]

                    new_labels.append(' '.join([str(x) for x in label]))

                    print(label)
                
                with open(file_path, 'w') as f:
                    for label in new_labels:
                        f.write(label + '\n')

fix_labels()


# Removing unnecessary labels


In [ ]:
def processing(files, root):
    for file in files:
        if file.endswith('.txt'):
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as f:
                lines = f.readlines()
            with open(file_path, 'w') as f:
                for line in lines:
                    if line.split()[0] != '5':
                        f.write(line) 

def remove_pcb():
    rendered_path = os.path.abspath('./datasets/rendered')
    aspl_path = os.path.abspath('./datasets/aspl_images')

    for root, _, files in os.walk(aspl_path):
        processing(files, root)
    
    for root, _, files in os.walk(rendered_path):
        processing(files, root)

remove_pcb()


# OpenCV

Applying transformation to certain images, especially on hue, brightness, and contrast

Additionnaly, maybe applying jpg compression to the images to simulate the real world scenario.
